## Sesión 4

En esta última sesión se propone realizar un ejercicio libre con el objetivo de mejorar lo máximo posible el acierto en la tarea MNIST. A modo de orientación se entiende por una buena tasa de acierto en esta tarea cuando obtenemos más de un 99% de acierto.

Para conseguir este resultado podréis probar pipelines que preprocesen los datos adecuadamente. Probar diferentes clasificadores e incluso combinación de los mismos.

In [2]:
## imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# ...
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier

In [6]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

In [7]:
# Partición de Datos
tam_test = 0.1
tam_val = 0.1
tam_train = 0.8

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=tam_test, random_state=23)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=tam_val/(tam_train + tam_test), random_state=23) 

Ejercicio: probar todo lo necesario para obtener un acc > 99%

In [13]:
## Ejercicio obtener acc>99%
clf1 = KNeighborsClassifier(n_neighbors=3, p=1, n_jobs=-1)
#clf2 = HistGradientBoostingClassifier(max_depth=64,max_bins=64)
clf2 = HistGradientBoostingClassifier()
clf3 = VotingClassifier(estimators=[('kn', clf1), ('hg', clf2)], n_jobs=-1)

In [ ]:
max = -1
for nc2 in [1,2,3,4,5]:
    clf7 = KNeighborsClassifier(n_neighbors=nc2, p=1, n_jobs=-1)
    y_pred = clf7.fit(X_train, y_train)
    acc = accuracy_score(y_val, y_pred.predict(X_val))
    print('La precisión de {0!s} es {1:.1%}'.format(clf7, acc))

    if acc>max:
        max = acc
        bestNc2 = nc2

In [8]:
max = -1
bestK = 5
for nc in [16, 32]:
    for k in range(bestK - 2, bestK + 3):
        clf4 = make_pipeline(PCA(n_components=nc), KNeighborsClassifier(n_neighbors=k, p=1, n_jobs=-1)).fit(X_train, y_train)
        acc = accuracy_score(y_val, clf4.predict(X_val))
        print('La precisión de {0!s} es {1:.1%}'.format(clf4, acc))

        if acc>max:
            max = acc
            bestNc = nc
            bestK = k

La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_jobs=-1, n_neighbors=3, p=1))]) es 96.0%
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_jobs=-1, n_neighbors=4, p=1))]) es 95.9%
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('kneighborsclassifier', KNeighborsClassifier(n_jobs=-1, p=1))]) es 95.9%
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_jobs=-1, n_neighbors=6, p=1))]) es 95.9%
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_jobs=-1, n_neighbors=7, p=1))]) es 95.9%
La precisión de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('kneighborsclassifier',
   

In [9]:
clf4 = make_pipeline(PCA(n_components=16), KNeighborsClassifier(n_neighbors=3, p=1, n_jobs=-1))
clf5 = make_pipeline(PCA(n_components=32), KNeighborsClassifier(n_neighbors=5, p=1, n_jobs=-1))

In [4]:
from xgboost import XGBClassifier

In [10]:
clf6 = XGBClassifier(
    n_estimators=100,
    subsample=0.75,
    colsample_bylevel=0.75,
    max_depth=8,
    random_state=23
)

clf6.fit(X_train, y_train)
y_pred = clf6.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print('La precisión de {0!s} es {1:.1%}'.format(clf6, acc))

La precisión de XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.75, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...) es 97.4%


In [11]:
clfFin = VotingClassifier(estimators=[('c1', clf1), ('c2', clf2), ('c4', clf4), ('c5', clf5), ('c6', clf6)], n_jobs=-1)

In [14]:
#for nc in [4,8,16,32]:
#clf = make_pipeline(PCA(n_components=32), clfFin)
clfFin.fit(X_train, y_train)
y_pred = clfFin.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print('La precisión de {0!s} es {1:.1%}'.format(clfFin, acc))

La precisión de VotingClassifier(estimators=[('c1',
                              KNeighborsClassifier(n_jobs=-1, n_neighbors=3,
                                                   p=1)),
                             ('c2', HistGradientBoostingClassifier()),
                             ('c4',
                              Pipeline(steps=[('pca', PCA(n_components=16)),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier(n_jobs=-1,
                                                                    n_neighbors=3,
                                                                    p=1))])),
                             ('c5',
                              Pipeline(steps=[('pca', PCA(n_components=32)),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier...
                                            gpu_id=None, grow_p